In [2]:
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language
from llama_index.core.node_parser import CodeSplitter
from llama_index.core import SimpleDirectoryReader
from vector_storing import vector_storing
from vector_load import vector_load


def github_code(url,repo_name):
    repo_path = ".\\{}\\".format(repo_name)
    repo = Repo.clone_from(url, to_path=repo_path)

c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
# repo_path

In [22]:
repo_name="FrugalGPT"
repo_url = "https://github.com/stanford-futuredata/FrugalGPT"
try:
    github_code("https://github.com/stanford-futuredata/FrugalGPT",repo_name)
except:
    pass

In [23]:
splitter = CodeSplitter('python')

documents = SimpleDirectoryReader("./{}".format(repo_name),recursive=True,required_exts=[".py", ".ipynb"]).load_data()

In [24]:
for i in documents:
    i.metadata['repo_name']=repo_name
    i.metadata['repo_link']=repo_url+'/blob/master/'+i.metadata['file_path'][i.metadata['file_path'].find(repo_name)+len(repo_name):]

documents

[Document(id_='27d65e76-90ec-480f-9894-880a887672cb', embedding=None, metadata={'file_path': 'd:\\hackathons\\code_philly\\FrugalGPT\\intro.ipynb', 'file_name': 'intro.ipynb', 'file_size': 49561, 'creation_date': '2024-04-20', 'last_modified_date': '2024-04-20', 'repo_name': 'FrugalGPT', 'repo_link': 'https://github.com/stanford-futuredata/FrugalGPT/blob/master/\\intro.ipynb'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='\n\n\nget_ipython().run_line_magic(\'load_ext\', \'autoreload\')\nget_ipython().run_line_magic(\'autoreload\', \'2\')\nimport sys, json, copy \nimport logging\nlogging.disable(logging.CRITICAL)\nsys.path.append("src/")\n\n\n# ## Setup\n# Next, let us set up the environment. Currently, FrugalGPT leverages LLM APIs from OpenAI (includi

In [25]:
documents[2].metadata

{'file_path': 'd:\\hackathons\\code_philly\\FrugalGPT\\intro.ipynb',
 'file_name': 'intro.ipynb',
 'file_size': 49561,
 'creation_date': '2024-04-20',
 'last_modified_date': '2024-04-20',
 'repo_name': 'FrugalGPT',
 'repo_link': 'https://github.com/stanford-futuredata/FrugalGPT/blob/master/\\intro.ipynb'}

In [26]:
nodes = splitter.get_nodes_from_documents(documents)

In [27]:
nodes[8].text

'# load data\ndev = FrugalGPT.loadcsvdata("data/HEADLINES/train.csv")\ndev = dev[0:10]\nprefix = open(\'config/prompt/HEADLINES/prefix_e8.txt\').read()\ndata = FrugalGPT.formatdata(dev,prefix)\n\n\n# Second, specify the budget per query, and then train the model. Warning: This can take a while on large datasets!\n\n#'

In [28]:
code_index = vector_storing("./embeddings/code","text-embedding-3-small","vector_store",nodes,"VectorStoreIndex")

In [29]:
code_query_engine = code_index.as_query_engine(similarity_top_k=15)

response = code_query_engine.query('Can I get the repo link of scoring code')

In [30]:
str(response)

'The repo link of the scoring code is https://github.com/stanford-futuredata/FrugalGPT/blob/master/\\src\\FrugalGPT\\scoring.py'

In [10]:
code_index.index_struct

IndexDict(index_id='3e80a948-d50e-456b-a292-27ffa6bd63a7', summary=None, nodes_dict={'7c382217-d342-4f00-a424-64ba3a7e4139': '7c382217-d342-4f00-a424-64ba3a7e4139', 'ff99b304-6be5-49c0-be17-52b0e30dbd99': 'ff99b304-6be5-49c0-be17-52b0e30dbd99', 'e1b33aab-2311-4f3a-80cd-513dce8a7ad0': 'e1b33aab-2311-4f3a-80cd-513dce8a7ad0', '275ea5c0-b034-43a6-abae-d12054c8dd1e': '275ea5c0-b034-43a6-abae-d12054c8dd1e', '111b6ae1-6746-400a-a5df-4397345b052e': '111b6ae1-6746-400a-a5df-4397345b052e', '729e1857-b9f5-4eb3-aa05-9f518faf9657': '729e1857-b9f5-4eb3-aa05-9f518faf9657', '5a74e21d-2abf-4798-8a5b-84d4a5bb7446': '5a74e21d-2abf-4798-8a5b-84d4a5bb7446', '441e6ee4-e27f-4df6-b5ae-760c1fede483': '441e6ee4-e27f-4df6-b5ae-760c1fede483', '939a55f2-3e1e-45f8-8be9-25ffbb505917': '939a55f2-3e1e-45f8-8be9-25ffbb505917', 'f7c643db-57f4-4eae-921e-ccecb66742c9': 'f7c643db-57f4-4eae-921e-ccecb66742c9', 'b882119b-bb08-4ff9-8d79-f3fb685ea129': 'b882119b-bb08-4ff9-8d79-f3fb685ea129', '2b632c1d-191f-4cd7-b23c-dba8334140

In [10]:
code_index = vector_load("./embeddings/code","text-embedding-3-small","vector_store","VectorStoreIndex")

code_query_engine = code_index.as_query_engine(similarity_top_k=15)

In [12]:
response = code_query_engine.query('can you give the code')

In [13]:
response

Response(response='Empty Response', source_nodes=[], metadata=None)

In [11]:
code_index.index_struct

IndexDict(index_id='8a427a3e-040c-445f-93f0-28c9e62acf5f', summary=None, nodes_dict={}, doc_id_dict={}, embeddings_dict={})

In [ ]:

    splitter = CodeSplitter('python')

    documents = SimpleDirectoryReader(r"D:/Akhil_Main_LLM_Code/Akhil_Main_LLM_Code/ASRAG/pages/RAG/code").load_data()
    nodes = splitter.get_nodes_from_documents(documents)

    vector_storing("./pages/RAG/embeddings/code","text-embedding-3-small","vector_store",nodes,"VectorStoreIndex")

In [ ]:
code_index = vector_load("./embeddings/code","text-embedding-3-small","vector_store","VectorStoreIndex")

    code_query_engine = code_index.as_query_engine(similarity_top_k=15,llm=model)


    code_tool = QueryEngineTool.from_defaults(
        query_engine=code_query_engine,
        description=(
            "Useful for answering code related queries and provide explanation in natural language along with code"
        ),
    )